# level2 
1.  提取topic，key_concept
2.  根据topic，key_concept，docs进行问题生成

In [ ]:
# 提取topic，key_concept
prompt_extract_question_level1 = """Here is an article crawl from the web, which our classifier has identified as having significant educational value for
    students learning math.
    As a senior **math** instructor, your task is to create **challenging computation-based math questions**. These
    questions should be suitable for various contexts, such as homework assignments, exams, interview preparations,
    classroom activities, competitions, and tutoring sessions while enhancing students’ reasoning and critical-thinking
    skills. Ensure that questions are **non-redundant**, precise, and engaging.
    ### Guidelines for Creating Computation-based Questions:
    1. **Assess Suitability**: If this article does not contain math-related content that can be used to generate engaging
    and solvable questions, please directly output "NOT SUITABLE for creating questions."
    2. **Generate Questions**: If the article is suitable for creating math questions, generate **1 to 5** questions
    based on the richness and depth of the article content. For articles covering multiple topics, aim to generate more
    questions to ensure coverage:
    - Each question must be solvable independently and should not rely on answers from previous questions.
    - If a question closely resembles the original text, append "<original_question>" at the end of the question.
    - If a question is newly created, append "<newly_created>" at the end of the question.
    3. **Content Alignment**: Your math questions must exclusively draw from the content of the article, ensuring
    they are directly aligned with the concepts presented.
    4. **Use Original Questions**: Use original questions from the article whenever possible. However, feel free to
    rephrase them for clarity and improved understanding.
    5. **Create New Questions**: Attempt to formulate **newly diverse and challenging questions** that explore
    different aspects of the content presented in the article.
    6. **Self-Contained Questions**: Ensure that each question is self-contained, meaning students do not need to
    read the article to answer them.
    7. **Logical Consistency**: Verify that the questions are **logically sound and directly aligned with the
    mathematical principles** in the article. You MUST minimize the use of **sub-questions**, unless they are
    essential to the problem’s complexity.
    8. **Clarity and Precision**:
    - Use precise and unambiguous language.
    - Write all mathematical expressions or formulas in LaTeX for clarity.
    - Clearly state all assumptions or conditions.
    - The answer should either be exact, or if not possible, then the question should clearly say the
    answer is only expected to be approximately correct.
    #### Article
    {{ text }}
    #### Output Format For each question, provide the following information:
    - **Question Content**: The actual math question.
    - **School Level**: Specify the school level (e.g., <elementary>, <middle_school>, <high_school>, <college>,
    <grad_school>, <competition>).
    - **Originality Tag**: Append "<original_question>" for original questions from the article or"<newly_created>"
    for newly created questions.
    Example Output:
    <Q1>
    Question: Content
    Orig_tag:<original_question>
    Level:<high_school>
    </Q1>
    <Q2>
    Question: Content
    Orig_tag:<newly_created>
    Level:<college>
    </Q2>"""
prompt_extract_topic = """Here is an article crawl from the web, which our classifier has identified as having significant educational value for
    students learning math.
    Your task is to analyze this article and extract educational materials, specifically focusing on topics and key
    concepts that can enhance students’ understanding of mathematics and improve their problem-solving skills.
    Pay special attention to uncommon but important mathematical concepts that are crucial for a deeper understanding.
    ## Tasks
    1. **Determine Educational Level:**
    - Identify the appropriate educational level for the article based on its content. Choose from the
    following options:
    - Primary School
    - Middle School
    - High School
    - College
    - Graduate School
    - Competition
    - Other
    2. **Identify Subject Area:**
    - Specify the primary subject area of mathematics to which the article belongs (e.g., Calculus,
    Geometry, Algebra, etc.).
    3. **Extract Topics and Key Concepts:**
    - **Topics:**
    - List **1 to 5** main topics covered in the article.
    - Use terms commonly recognized in academia or industry.
    - **Key Concepts:**
    - For each identified topic, list **5 to 20** related key concepts.
    - Ensure these concepts are clearly articulated using standard academic or industry terms.
    ## Guidelines:
    - **Terminology:** Use precise and widely recognized academic or industry terminology for subjects, topics, and
    key concepts to maintain consistency and clarity.
    - **Educational Level Selection:** If appropriate, restrict the educational level to one of the following: "Primary
    School", "Middle School", "High School", "College", "Graduate School", or "Competition" to ensure accurate
    categorization.
    ## Text
    {{ text }}
    ## Output Format
    <level>Educational Level</level>
    <subject>Subject Area</subject>
    <topic> Topics:
    1. topic 1
    2. topic 2
    </topic>
    <key_concept>
    Key Concepts:
    1. topic 1:
    1.1. key concept
    1.2. key concept
    ...
    2. topic 2:
    2.1. key concept
    ... ...
    </key_concept>
    ## Output"""
prompt_extract_question = """As a senior **math** instructor, your task is to create **diverse and challenging computation-based math
    questions**. These questions should demonstrate the application of the provided topics and key concepts while
    enhancing students’ reasoning and critical-thinking skills. Ensure that questions are **non-redundant**, precise,
    and engaging.
    ### Guidelines for Creating Diverse and Challenging Computation-based Questions:
    1. **Concept Selection**:
    - Randomly select **up to 2-3 distinct key concepts** from the provided list for each question.
    - Ensure **broad coverage** of the provided concepts across the generated questions, avoiding over-reliance
    on a limited subset of concepts.
    - Avoid repeating the same **concept combinations** or **computational approach** across questions.
    2. **Diversity and Challenge**:
    - Ensure that each question explores **different combinations of key concepts** and is **sufficiently
    challenging** (e.g., requiring multi-step computations, integrating real-world scenarios, involving abstract or
    advanced reasoning.).
    3. **Clarity and Precision**:
    - Verify that the questions are **logically sound**.
    - Use precise and unambiguous language.
    - Write all mathematical expressions or formulas in LaTeX for clarity.
    - Clearly state all assumptions or conditions.
    4. **Reference Material**:
    - Use the provided **reference article** as a source of inspiration for generating **unique, diverse, and
    challenging questions**.
    - The reference material is intended to:
    - Supplement the concept list by introducing **novel perspectives**, **contexts**, or **applications**.
    - Help create questions that are **more complex, realistic, or uncommon** in traditional teaching scenarios.
    - Serve as a resource to craft **real-world scenarios** or **abstract extensions** beyond the given concepts.
    5. **Output Diversity**:
    - Create between **1 to 5 questions**.
    - Ensure each question is unique in **structure**, **approach**, and **concept usage**.
    - Minimize the use of **sub-questions**, unless they are essential to the problem’s complexity.
    - The answer should either be exact, or if not possible, then the question should clearly say the answer is only
    expected to be approximately correct.
    ### Inputs:
    - **Article**:
    {{ text }}
    - **Concept List**:
    {{ concept }}
    #### Output Format:
    <Q1>
    Selected Concepts: [Only insert 2-3 concepts here]
    Question: [Only insert question here]
    </Q1>
    <Q2>
    Selected Concepts: [Only insert 2-3 concepts here]
    Question: [Only insert question here]
    </Q2>"""

In [5]:
from utils.topic_concepts import trans_chunk4
chunks_path = "outputs_chunks/article_chunks11.json"
chunk4_path = "outputs_chunk4/article_chunkf03.json"
trans_chunk4(chunks_path, chunk4_path)

Chunk4 saved to outputs_chunk4/article_chunkf03.json
done 400×640象素液晶显示的研制_杨兆芳_llm_correct.md chunk4.
Chunk4 saved to outputs_chunk4/article_chunkf03.json
done 5G时代新型先进显示技术发...—访中国科学院院士欧阳钟灿_张莹_llm_correct.md chunk4.
Chunk4 saved to outputs_chunk4/article_chunkf03.json
done 5种液晶显示优劣谈_llm_correct.md chunk4.


In [6]:
from utils.common_utils import build_openai_client_chat
from utils.topic_concepts import gen_topics
chunk4_path = "outputs_chunk4/article_chunkf03.json"
topics_path = "outputs_topics/article_topics03.json"
chat_model = build_openai_client_chat()
gen_topics(chunk4_path, topics_path, chat_model)

Topicing:  50%|█████     | 1/2 [00:36<00:36, 36.10s/file]

Topics saved to outputs_topics/article_topics03.json


Topicing: 100%|██████████| 2/2 [00:39<00:00, 19.87s/file]


Topics saved to outputs_topics/article_topics03.json
done 400×640象素液晶显示的研制_杨兆芳_llm_correct.md topics.


Topicing:  17%|█▋        | 1/6 [00:45<03:47, 45.48s/file]

...

Topics saved to outputs_topics/article_topics03.json


Topicing: 100%|██████████| 6/6 [02:26<00:00, 24.39s/file]


Topics saved to outputs_topics/article_topics03.json
done 5G时代新型先进显示技术发...—访中国科学院院士欧阳钟灿_张莹_llm_correct.md topics.


Topicing: 100%|██████████| 1/1 [00:39<00:00, 39.41s/file]

Topics saved to outputs_topics/article_topics03.json
done 5种液晶显示优劣谈_llm_correct.md topics.


In [1]:
output = """<Q>
Selected Concepts: [Only insert 2-3 concepts here]
Question: [Only insert question here]
</Q>
<Q>
Selected Concepts: [Only insert 2-3 concepts here]
Question: [Only insert question here]
</Q>"""
def parse_questions(questions):
    questions = questions.split("</Q>")[:-1]
    print(f"question: {questions}")
    questions = [q.split("Question: ")[1] for q in questions]
    print(f"question: {questions}")
    questions = [q.split("Selected Concepts: ")[0] for q in questions]
    print(f"question: {questions}")
    questions = [q.strip() for q in questions]
    return questions
questions = parse_questions(output)
questions

question: ['<Q>\nSelected Concepts: [Only insert 2-3 concepts here]\nQuestion: [Only insert question here]\n', '\n<Q>\nSelected Concepts: [Only insert 2-3 concepts here]\nQuestion: [Only insert question here]\n']
question: ['[Only insert question here]\n', '[Only insert question here]\n']
question: ['[Only insert question here]\n', '[Only insert question here]\n']


['[Only insert question here]', '[Only insert question here]']

In [5]:
import json
output_questions = """[
            {
                "concepts": ["Only insert 2-3 concepts here"],
            "question": "Only insert question here"
        },
        {
            "concept": ["Only insert 2-3 concepts here"],
            "question": "Only insert question here"
        }
    ]"""
# 将上述输出进行解析，并返回
output_questions = json.loads(output_questions)
output_questions

[{'concepts': ['Only insert 2-3 concepts here'],
  'question': 'Only insert question here'},
 {'concept': ['Only insert 2-3 concepts here'],
  'question': 'Only insert question here'}]

In [1]:
from utils.common_utils import build_openai_client_chat
from utils.topic_concepts import gen_questions_with_topic
topics_path = "outputs_topics/article_topics03.json"
question_path = "outputs_questions/article_question_with_topic04.json"
chat_model = build_openai_client_chat()
gen_questions_with_topic(topics_path, question_path, chat_model)

Questioning:  50%|█████     | 1/2 [01:24<01:24, 84.74s/file]

Questions saved to outputs_questions/article_question_with_topic04.json


Questioning: 100%|██████████| 2/2 [02:11<00:00, 65.52s/file]


Questions saved to outputs_questions/article_question_with_topic04.json
done 400×640象素液晶显示的研制_杨兆芳_llm_correct.md questions.


Questioning:  17%|█▋        | 1/6 [01:24<07:04, 84.84s/file]

...

Questions saved to outputs_questions/article_question_with_topic04.json


Questioning: 100%|██████████| 6/6 [04:19<00:00, 43.20s/file]


Questions saved to outputs_questions/article_question_with_topic04.json
done 5G时代新型先进显示技术发...—访中国科学院院士欧阳钟灿_张莹_llm_correct.md questions.


Questioning: 100%|██████████| 1/1 [03:22<00:00, 202.57s/file]

Questions saved to outputs_questions/article_question_with_topic04.json
done 5种液晶显示优劣谈_llm_correct.md questions.


In [1]:
from utils.common_utils import build_openai_client_chat
from utils.answer_generation import gen_answer
questions_path = "outputs_questions/article_question_with_topic04.json"
answers_path = "outputs_answers/article_answers_02.json"
chat_model = build_openai_client_chat()
gen_answer(questions_path, chat_model, answers_path)

Answering:  25%|██▌       | 1/4 [01:04<03:14, 64.77s/file]

Answers saved to outputs_answers/article_answers_02.json


Answering:  50%|█████     | 2/4 [01:16<01:07, 33.51s/file]

Answers saved to outputs_answers/article_answers_02.json


Answering:  75%|███████▌  | 3/4 [02:06<00:41, 41.11s/file]

...

Answers saved to outputs_answers/article_answers_02.json
done 5G时代新型先进显示技术发...—访中国科学院院士欧阳钟灿_张莹_llm_correct.md answers.


Answering:  50%|█████     | 1/2 [01:56<01:56, 116.31s/file]

Answers saved to outputs_answers/article_answers_02.json


Answering: 100%|██████████| 2/2 [02:14<00:00, 67.12s/file] 

Answers saved to outputs_answers/article_answers_02.json
done 5种液晶显示优劣谈_llm_correct.md answers.


In [9]:
import demjson
questions = """```json
[
    {
        "concepts": ["RGB subpixel addressing", "Current mirror matching", "Dynamic power management"],
        "question": "A PM-OLED driver must produce 65,000 colors using 8-bit RGB subpixel intensities. Calculate the required number of matched current mirrors per column driver if each RGB subpixel requires 256 discrete current levels, considering a maximum allowable current mismatch of 0.4% between adjacent intensity levels. How does dynamic voltage scaling in power management affect the mirror matching requirements during screen saver activation?"
    },
    {
        "concepts": ["Parasitic capacitance compensation", "Pre-charge circuit optimization", "Energy-efficient driving waveforms"],
        "question": "For a 96×64 OLED panel with 15pF column line capacitance, determine the optimal pre-charge voltage (V_pre) and timing duration (t_pre) required to achieve 90% settling within 1μs frame time. The driver uses a switched-resistor precharge circuit with 5kΩ impedance. How would triangular vs rectangular driving waveforms impact both capacitance compensation effectiveness and power consumption?"
    },
    {
        "concepts": ["256-level contrast adjustment", "Negative feedback current stabilization", "Leakage current reduction"],
        "question": "A contrast adjustment DAC uses negative feedback to maintain 1.2mA full-scale current with ±0.25% stability. Calculate the required open-loop gain of the error amplifier when process variations cause 8% resistance mismatch in current mirrors. How does subthreshold biasing in standby mode affect the DAC's startup settling characteristics?"
    },
    {
        "concepts": ["Screen saver implementation", "Display uniformity maintenance", "Battery-operated system optimization"],
        "question": "A portable medical device displays static ECG waveforms 60% of the time. Compare total system current consumption between always-on and screen saver modes (30% display duty cycle) when the 96×64 panel draws 18mA active current and 150μA standby current. What pixel refresh rate reduction strategy would maintain acceptable waveform visibility while maximizing battery life?"
    },
    {
        "concepts": ["Signal timing synchronization", "Analog current source design", "Portable device display integration"],
        "question": "Calculate the maximum tolerable clock skew between row and column drivers in a 96×64 PM-OLED system operating at 85Hz refresh rate, given that pixel current must stabilize within 12% of the 1.8μs line time. How does temperature-induced mobility variation in analog current sources affect this timing margin across -20°C to 60°C operation?"
    }
]
```"""

try:
    output_questions = demjson.decode(questions, strict=False)
except demjson.JSONDecodeError as e:
    print(f"Demjson decode error: {e}")
    output_questions = []

Demjson decode error: Can not decode value starting with character '`'


In [ ]:
import re
import regex
import json

def extract_and_parse_json(text):
    # 使用 regex 提取 JSON 部分
    match = regex.search(r'\$$(?:[^\$$]|(?R))*\$$|{(?:[^{]|(?R))*}|\[(?:[^\$$]|(?R))*]', text, flags=regex.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except json.JSONDecodeError as e:
            print(f"无法解析提取的 JSON 内容: {e}")
    else:
        print("未找到有效的 JSON 内容。")
    return None
questions = """```json
[
    {
        "concepts": ["RGB subpixel addressing", "Current mirror matching", "Dynamic power management"],
        "question": "A PM-OLED driver must produce 65,000 colors using 8-bit RGB subpixel intensities. Calculate the required number of matched current mirrors per column driver if each RGB subpixel requires 256 discrete current levels, considering a maximum allowable current mismatch of 0.4% between adjacent intensity levels. How does dynamic voltage scaling in power management affect the mirror matching requirements during screen saver activation?"
    },
    {
        "concepts": ["Parasitic capacitance compensation", "Pre-charge circuit optimization", "Energy-efficient driving waveforms"],
        "question": "For a 96×64 OLED panel with 15pF column line capacitance, determine the optimal pre-charge voltage (V_pre) and timing duration (t_pre) required to achieve 90% settling within 1μs frame time. The driver uses a switched-resistor precharge circuit with 5kΩ impedance. How would triangular vs rectangular driving waveforms impact both capacitance compensation effectiveness and power consumption?"
    },
    {
        "concepts": ["256-level contrast adjustment", "Negative feedback current stabilization", "Leakage current reduction"],
        "question": "A contrast adjustment DAC uses negative feedback to maintain 1.2mA full-scale current with ±0.25% stability. Calculate the required open-loop gain of the error amplifier when process variations cause 8% resistance mismatch in current mirrors. How does subthreshold biasing in standby mode affect the DAC's startup settling characteristics?"
    },
    {
        "concepts": ["Screen saver implementation", "Display uniformity maintenance", "Battery-operated system optimization"],
        "question": "A portable medical device displays static ECG waveforms 60% of the time. Compare total system current consumption between always-on and screen saver modes (30% display duty cycle) when the 96×64 panel draws 18mA active current and 150μA standby current. What pixel refresh rate reduction strategy would maintain acceptable waveform visibility while maximizing battery life?"
    },
    {
        "concepts": ["Signal timing synchronization", "Analog current source design", "Portable device display integration"],
        "question": "Calculate the maximum tolerable clock skew between row and column drivers in a 96×64 PM-OLED system operating at 85Hz refresh rate, given that pixel current must stabilize within 12% of the 1.8μs line time. How does temperature-induced mobility variation in analog current sources affect this timing margin across -20°C to 60°C operation?"
    }
]
```"""
# 使用通用函数解析
parsed_data = extract_and_parse_json(questions)
if parsed_data:
    print(parsed_data)

[{'concepts': ['RGB subpixel addressing', 'Current mirror matching', 'Dynamic power management'], 'question': 'A PM-OLED driver must produce 65,000 colors using 8-bit RGB subpixel intensities. Calculate the required number of matched current mirrors per column driver if each RGB subpixel requires 256 discrete current levels, considering a maximum allowable current mismatch of 0.4% between adjacent intensity levels. How does dynamic voltage scaling in power management affect the mirror matching requirements during screen saver activation?'}, {'concepts': ['Parasitic capacitance compensation', 'Pre-charge circuit optimization', 'Energy-efficient driving waveforms'], 'question': 'For a 96×64 OLED panel with 15pF column line capacitance, determine the optimal pre-charge voltage (V_pre) and timing duration (t_pre) required to achieve 90% settling within 1μs frame time. The driver uses a switched-resistor precharge circuit with 5kΩ impedance. How would triangular vs rectangular driving wavef

In [6]:
questions1 = """

[
    {
        "concepts": ["Operational amplifiers (Op-Amps) in voltage-to-current conversion", "Subthreshold conduction exponential model"],
        "question": "A voltage-to-current (V/I) converter uses a bandgap reference voltage \( V_{\mathrm{bg}} = 1.2~\mathrm{V} \) and an external resistor \( R_0 = 15~\mathrm{k\Omega} \). The resulting current drives a MOSFET operating in the subthreshold region with parameters \( I_{\mathrm{d0}} = 10~\mathrm{nA} \), \( \zeta = 1.5 \), and \( V_{\mathrm{th}} = 0.4~\mathrm{V} \). Calculate the required \( W/L \) ratio to achieve a drain current of \( 75~\mathrm{nA} \) at \( V_{\mathrm{gs}} = 0.45~\mathrm{V} \). Assume thermal voltage \( V_{\mathrm{th,thermal}} = 26~\mathrm{mV} \)."
    },
    {
        "concepts": ["N-bit current-steering DAC architecture", "Binary-weighted current sources"],
        "question": "A 5-bit brightness control DAC uses binary-weighted current sources with a maximum output current of \( 680~\mathrm{\mu A} \). If the MSB current source corresponds to \( 16 \times I_{\mathrm{LSB}} \), determine (a) the LSB current value and (b) the total current range when all bits are activated. Verify if this matches the specified \( 5~\mathrm{\mu A} \) to \( 680~\mathrm{\mu A} \) range mentioned in the system."
    },
    {
        "concepts": ["Charge pump circuits for negative voltage generation", "Power dissipation trade-offs in conduction regimes"],
        "question": "A charge pump generates a -9V cathode voltage with 85% efficiency. If the display operates at 30 MHz with a total switched capacitance of \( 150~\mathrm{pF} \) and draws \( 2.8~\mathrm{mA} \) from a 3.3V supply, calculate the total power dissipation considering both switching losses and DC current consumption."
    },
    {
        "concepts": ["Non-overlapping clock circuits for dynamic power reduction", "Frequency response optimization"],
        "question": "A non-overlapping clock circuit operates at 25 MHz with 1.2V swing and 5 pF load capacitance per phase. Calculate the dynamic power saved per cycle compared to a standard clock, assuming the non-overlapping design eliminates 80% of the crowbar current that normally occurs during 15% of the transition period in a conventional clock."
    },
    {
        "concepts": ["Saturation region square law relationship", "Threshold voltage mismatch sensitivity"],
        "question": "A saturated nMOS constant current source must maintain \( I_d = 120~\mathrm{\mu A} \) despite \( \pm 8\% \) \( V_{\mathrm{th}} \) variation. Using the square law equation \( I_d = \frac{1}{2} \mu_n C_{\mathrm{ox}} (W/L) (V_{\mathrm{gs}} - V_{\mathrm{th}})^2 \), determine the minimum \( V_{\mathrm{gs}} \) that guarantees \( I_d \geq 100~\mathrm{\mu A} \) under worst-case threshold voltage conditions. Assume \( \mu_n C_{\mathrm{ox}} (W/L) = 0.5~\mathrm{mA/V^2} \)."
    }
]"""
# 使用通用函数解析
parsed_data = extract_and_parse_json(questions1)
if parsed_data:
    print(parsed_data)

无法解析提取的 JSON 内容: Invalid \escape: line 4 column 92 (char 235)


In [1]:
questions1 = """```json
[
    {
        "concepts": ["RGB subpixel addressing", "Current mirror matching", "Dynamic power management"],
        "question": "A PM-OLED driver must produce 65,000 colors using 8-bit RGB subpixel intensities. Calculate the required number of matched current mirrors per column driver if each RGB subpixel requires 256 discrete current levels, considering a maximum allowable current mismatch of 0.4% between adjacent intensity levels. How does dynamic voltage scaling in power management affect the mirror matching requirements during screen saver activation?"
    },
    {
        "concepts": ["Parasitic capacitance compensation", "Pre-charge circuit optimization", "Energy-efficient driving waveforms"],
        "question": "For a 96×64 OLED panel with 15pF column line capacitance, determine the optimal pre-charge voltage (V_pre) and timing duration (t_pre) required to achieve 90% settling within 1μs frame time. The driver uses a switched-resistor precharge circuit with 5kΩ impedance. How would triangular vs rectangular driving waveforms impact both capacitance compensation effectiveness and power consumption?"
    },
    {
        "concepts": ["256-level contrast adjustment", "Negative feedback current stabilization", "Leakage current reduction"],
        "question": "A contrast adjustment DAC uses negative feedback to maintain 1.2mA full-scale current with ±0.25% stability. Calculate the required open-loop gain of the error amplifier when process variations cause 8% resistance mismatch in current mirrors. How does subthreshold biasing in standby mode affect the DAC's startup settling characteristics?"
    },
    {
        "concepts": ["Screen saver implementation", "Display uniformity maintenance", "Battery-operated system optimization"],
        "question": "A portable medical device displays static ECG waveforms 60% of the time. Compare total system current consumption between always-on and screen saver modes (30% display duty cycle) when the 96×64 panel draws 18mA active current and 150μA standby current. What pixel refresh rate reduction strategy would maintain acceptable waveform visibility while maximizing battery life?"
    },
    {
        "concepts": ["Signal timing synchronization", "Analog current source design", "Portable device display integration"],
        "question": "Calculate the maximum tolerable clock skew between row and column drivers in a 96×64 PM-OLED system operating at 85Hz refresh rate, given that pixel current must stabilize within 12% of the 1.8μs line time. How does temperature-induced mobility variation in analog current sources affect this timing margin across -20°C to 60°C operation?"
    }
]
```"""
questions2 = """

[
    {
        "concepts": ["Operational amplifiers (Op-Amps) in voltage-to-current conversion", "Subthreshold conduction exponential model"],
        "question": "A voltage-to-current (V/I) converter uses a bandgap reference voltage \( V_{\mathrm{bg}} = 1.2~\mathrm{V} \) and an external resistor \( R_0 = 15~\mathrm{k\Omega} \). The resulting current drives a MOSFET operating in the subthreshold region with parameters \( I_{\mathrm{d0}} = 10~\mathrm{nA} \), \( \zeta = 1.5 \), and \( V_{\mathrm{th}} = 0.4~\mathrm{V} \). Calculate the required \( W/L \) ratio to achieve a drain current of \( 75~\mathrm{nA} \) at \( V_{\mathrm{gs}} = 0.45~\mathrm{V} \). Assume thermal voltage \( V_{\mathrm{th,thermal}} = 26~\mathrm{mV} \)."
    },
    {
        "concepts": ["N-bit current-steering DAC architecture", "Binary-weighted current sources"],
        "question": "A 5-bit brightness control DAC uses binary-weighted current sources with a maximum output current of \( 680~\mathrm{\mu A} \). If the MSB current source corresponds to \( 16 \times I_{\mathrm{LSB}} \), determine (a) the LSB current value and (b) the total current range when all bits are activated. Verify if this matches the specified \( 5~\mathrm{\mu A} \) to \( 680~\mathrm{\mu A} \) range mentioned in the system."
    },
    {
        "concepts": ["Charge pump circuits for negative voltage generation", "Power dissipation trade-offs in conduction regimes"],
        "question": "A charge pump generates a -9V cathode voltage with 85% efficiency. If the display operates at 30 MHz with a total switched capacitance of \( 150~\mathrm{pF} \) and draws \( 2.8~\mathrm{mA} \) from a 3.3V supply, calculate the total power dissipation considering both switching losses and DC current consumption."
    },
    {
        "concepts": ["Non-overlapping clock circuits for dynamic power reduction", "Frequency response optimization"],
        "question": "A non-overlapping clock circuit operates at 25 MHz with 1.2V swing and 5 pF load capacitance per phase. Calculate the dynamic power saved per cycle compared to a standard clock, assuming the non-overlapping design eliminates 80% of the crowbar current that normally occurs during 15% of the transition period in a conventional clock."
    },
    {
        "concepts": ["Saturation region square law relationship", "Threshold voltage mismatch sensitivity"],
        "question": "A saturated nMOS constant current source must maintain \( I_d = 120~\mathrm{\mu A} \) despite \( \pm 8\% \) \( V_{\mathrm{th}} \) variation. Using the square law equation \( I_d = \frac{1}{2} \mu_n C_{\mathrm{ox}} (W/L) (V_{\mathrm{gs}} - V_{\mathrm{th}})^2 \), determine the minimum \( V_{\mathrm{gs}} \) that guarantees \( I_d \geq 100~\mathrm{\mu A} \) under worst-case threshold voltage conditions. Assume \( \mu_n C_{\mathrm{ox}} (W/L) = 0.5~\mathrm{mA/V^2} \)."
    }
]"""

In [ ]:
import json
import demjson
def clean_and_parse(json_str):
    # 去除 Markdown 的 ```json 和 ```
    cleaned = json_str.strip().removeprefix('```json').removesuffix('```').strip()
    try:
        return demjson.decode(cleaned)
    except demjson.JSONDecodeError as e:
        print(f"JSON 解码错误: {e}")
        return None
# 调用示例
parsed_data = clean_and_parse(questions1)
if parsed_data:
    print(parsed_data)
parsed_data = clean_and_parse(questions2)
if parsed_data:
    print(parsed_data)

[{'concepts': ['RGB subpixel addressing', 'Current mirror matching', 'Dynamic power management'], 'question': 'A PM-OLED driver must produce 65,000 colors using 8-bit RGB subpixel intensities. Calculate the required number of matched current mirrors per column driver if each RGB subpixel requires 256 discrete current levels, considering a maximum allowable current mismatch of 0.4% between adjacent intensity levels. How does dynamic voltage scaling in power management affect the mirror matching requirements during screen saver activation?'}, {'concepts': ['Parasitic capacitance compensation', 'Pre-charge circuit optimization', 'Energy-efficient driving waveforms'], 'question': 'For a 96×64 OLED panel with 15pF column line capacitance, determine the optimal pre-charge voltage (V_pre) and timing duration (t_pre) required to achieve 90% settling within 1μs frame time. The driver uses a switched-resistor precharge circuit with 5kΩ impedance. How would triangular vs rectangular driving wavef

In [1]:
import torch
print(torch.__version__)  # 应该能正常执行

2.5.1


In [ ]:
str1 = "asd"
print(f"dddd {'aa' if str1 == 'asd' else ''}")